In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import slim
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
inputs = x_train[np.where(y_train==1)[0],:,:,:]
x_test = x_test[np.where(y_test==1)[0],:,:,:]
inputs = inputs.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [3]:
with tf.variable_scope('W', reuse=tf.AUTO_REUSE) as W:
    W = slim.variable('w', shape=[7,7,3,3],
                 initializer = tf.truncated_normal_initializer(stddev=0.1),
                 device='/CPU:0')
print(W)

<tf.Variable 'W/w:0' shape=(7, 7, 3, 3) dtype=float32_ref>


# 1. 합성곱 하나씩 구현하기

In [5]:
with tf.variable_scope('net1', reuse=tf.AUTO_REUSE):
    print(inputs.shape)
    net = slim.conv2d(inputs, 64, [3, 3], padding='SAME',
                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005), scope='conv1_1')
    print(net)    
    net = slim.conv2d(net, 64, [3, 3], 1, padding='SAME',
                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005), scope='conv1_2')
    print(net)    
    net = slim.conv2d(net, 64, [11, 11], 1, padding='SAME',
                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005), scope='conv1_3')
    print(net)    


(5000, 32, 32, 3)
Tensor("net1/conv1_1/Relu:0", shape=(5000, 32, 32, 64), dtype=float32)
Tensor("net1/conv1_2/Relu:0", shape=(5000, 32, 32, 64), dtype=float32)
Tensor("net1/conv1_3/Relu:0", shape=(5000, 32, 32, 64), dtype=float32)


# 2. 합성곱 하나씩 구현하기(간단히 Default로)

In [5]:
with tf.variable_scope('net2', reuse=tf.AUTO_REUSE):
    net = slim.conv2d(inputs, 256, [3, 3], scope='conv2_1');print(net);
    net = slim.conv2d(net, 256, [3, 3], scope='conv2_2');print(net);
    net = slim.conv2d(net, 256, [3, 3], scope='conv2_3');print(net);
    net = slim.max_pool2d(net, [2, 2], scope='pool2');print(net);

Tensor("net2/conv2_1/Relu:0", shape=(5000, 32, 32, 256), dtype=float32)
Tensor("net2/conv2_2/Relu:0", shape=(5000, 32, 32, 256), dtype=float32)
Tensor("net2/conv2_3/Relu:0", shape=(5000, 32, 32, 256), dtype=float32)
Tensor("net2/pool2/MaxPool:0", shape=(5000, 16, 16, 256), dtype=float32)


# 3. 합성곱 반복하기(동일형태로만 가능)

In [6]:
with tf.variable_scope('net3', reuse=tf.AUTO_REUSE):
    net = slim.repeat(inputs, 5, slim.conv2d, 128, [3,3], scope='conv3')
    print(net)

Tensor("net3/conv3/conv3_5/Relu:0", shape=(5000, 32, 32, 128), dtype=float32)


# 4. 합성곱 순서대로 간단히 수행하기

In [7]:
with tf.variable_scope('net4', reuse=tf.AUTO_REUSE):
    net = slim.stack(net, slim.conv2d, [(64, [3,3]), (64, [1,1]), (128, [3,3]), (128, [1,1])])
    print(net)

Tensor("net4/Stack/convolution2d_4/Relu:0", shape=(5000, 32, 32, 128), dtype=float32)


# 5. 합성곱의 variable을 공통으로 적용하기

In [4]:
with tf.variable_scope('net5', reuse=tf.AUTO_REUSE):
    with slim.arg_scope([slim.conv2d], padding='SAME',
                          weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                          weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.conv2d(inputs, 64, [11, 11], scope='conv5_1')
        net = slim.conv2d(net, 128, [11, 11], padding='VALID', scope='conv5_2')
        net = slim.conv2d(net, 256, [11, 11], scope='conv5_3')

# 6. VGG 구현하기

In [9]:
with tf.variable_scope('net6', reuse=tf.AUTO_REUSE):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.relu,
                      weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1');print(net);
        net = slim.max_pool2d(net, [2, 2], scope='pool1');print(net);
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2');print(net);
        net = slim.max_pool2d(net, [2, 2], scope='pool2');print(net);
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3');print(net);
        net = slim.max_pool2d(net, [2, 2], scope='pool3');print(net);
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4');print(net);
        net = slim.max_pool2d(net, [2, 2], scope='pool4');print(net);
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5');print(net);
        net = slim.max_pool2d(net, [2, 2], scope='pool5');print(net);
        net = slim.fully_connected(net, 4096, scope='fc6');print(net);
        net = slim.dropout(net, 0.5, scope='dropout6');print(net);
        net = slim.fully_connected(net, 4096, scope='fc7');print(net);
        net = slim.dropout(net, 0.5, scope='dropout7');print(net);
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc8');print(net);


Tensor("net6/conv1/conv1_2/Relu:0", shape=(5000, 32, 32, 64), dtype=float32)
Tensor("net6/pool1/MaxPool:0", shape=(5000, 16, 16, 64), dtype=float32)
Tensor("net6/conv2/conv2_2/Relu:0", shape=(5000, 16, 16, 128), dtype=float32)
Tensor("net6/pool2/MaxPool:0", shape=(5000, 8, 8, 128), dtype=float32)
Tensor("net6/conv3/conv3_3/Relu:0", shape=(5000, 8, 8, 256), dtype=float32)
Tensor("net6/pool3/MaxPool:0", shape=(5000, 4, 4, 256), dtype=float32)
Tensor("net6/conv4/conv4_3/Relu:0", shape=(5000, 4, 4, 512), dtype=float32)
Tensor("net6/pool4/MaxPool:0", shape=(5000, 2, 2, 512), dtype=float32)
Tensor("net6/conv5/conv5_3/Relu:0", shape=(5000, 2, 2, 512), dtype=float32)
Tensor("net6/pool5/MaxPool:0", shape=(5000, 1, 1, 512), dtype=float32)
Tensor("net6/fc6/Relu:0", shape=(5000, 1, 1, 4096), dtype=float32)
Tensor("net6/dropout6/dropout/mul:0", shape=(5000, 1, 1, 4096), dtype=float32)
Tensor("net6/fc7/Relu:0", shape=(5000, 1, 1, 4096), dtype=float32)
Tensor("net6/dropout7/dropout/mul:0", shape=(500